In [37]:
from itertools import chain
from time import time

import pandas as pd
from src.models.kaggle_model3 import KaggleModel3
from thefuzz import fuzz
from thefuzz import process

In [2]:
validation_data = pd.read_pickle("../data/kaggle/validation_dataframe.pkl")

In [38]:
start = time()
output = KaggleModel3().inference_dataframe(
    {"params": "../models/kagglemodel3/params.txt"}, validation_data
)
total = time() - start

In [4]:
output

,ids,text,labels,KaggleModel3
0,7bfd8bb51,"[{'section_title': 'Abstract', 'text': 'Abstra...",dbgap,
1,7d3e31302,"[{'section_title': 'Abstract', 'text': 'Despit...",sass|sass data|school and staffing sass data|s...,
2,3644f959a,"[{'section_title': 'Highlights', 'text': 'Thro...",foodaps|national household food acquisition an...,Agricultural Resources Management Survey|ARMS
3,ed3527cf3,[{'section_title': 'INTRODUCTION AND STATEMENT...,database of genotypes and phenotypes|database ...,
4,236061129,"[{'section_title': 'Abstract', 'text': 'Highli...",dbgap,
...,...,...,...,...
8058,3cb8b0e09,"[{'section_title': 'Abstract', 'text': 'Aims/h...",database of genotypes and phenotypes|database ...,
8059,324fe1310,"[{'section_title': '', 'text': 'Neuroblastoma ...",dbgap,
8060,39e66a274,"[{'section_title': 'Introduction', 'text': 'Ex...",1000 genomes project|1000 genomes project 1000...,
8061,c411b1b6c,"[{'section_title': 'Abstract', 'text': 'Object...",dbgap|american cancer society cancer preventio...,


In [25]:
def calculate_statistics(row):
    predictions = list(set(row["model_prediction"].split("|")))
    labels = row["labels"].split("|")

    label_values = ["FN" for i in range(len(labels))]
    false_positives = []

    if len(predictions[0]) > 0:

        for i, prediction in enumerate(predictions):
            lbl_match = process.extractOne(prediction, labels, score_cutoff=50)
            if lbl_match is not None:
                label_values[labels.index(lbl_match[0])] = "TP"
            else:
                false_positives.append(prediction)

    output_statistics = {
        "labels": labels + false_positives,
        "stats": label_values + ["FP" for _ in range(len(false_positives))],
    }

    return output_statistics

In [26]:
output["statistics"] = output.apply(calculate_statistics, axis=1)

In [46]:
all_labels = list(chain(*list(map(lambda x: x["labels"], output["statistics"].values))))
global_stats = list(
    chain(*list(map(lambda x: x["stats"], output["statistics"].values)))
)
print(
    "Precision: ",
    global_stats.count("TP") / (global_stats.count("TP") + global_stats.count("FP")),
)
print(
    "Recall: ",
    global_stats.count("TP") / (global_stats.count("TP") + global_stats.count("FN")),
)
print(
    f"True Positive Count -- total: {global_stats.count('TP')} average: {global_stats.count('TP')/len(output)}"
)
print(f"Time -- total: {total:.2f}s average: {total/len(output):.2f}s")

Precision:  0.9342453662842012
Recall:  0.059327971302861306
True Positive Count -- total: 2117 average: 0.26255736078382735
Time -- total: 49.73s average: 0.01s


In [45]:
stats_df = pd.DataFrame({"labels": all_labels, "stats": global_stats})
stats_df.loc[stats_df["stats"] == "FN", :].groupby("labels").count().sort_values(
    "stats", ascending=False
)

,stats
labels,
dbgap,3712
database of genotypes and phenotypes,1338
program for international student assessment,1010
database of genotypes and phenotypes dbgap,836
schools and staffing survey,436
...,...
genome wide association study for bronchopulmonary dysplasia,1
genome wide association studies of psv,1
genome wide association studies of prematurity and its complications,1
